# iPython notebook for simulating visibility dataset  (by Anna Scaife)

This notebook uses code snippets from the crocodile python examples (https://github.com/SKA-ScienceDataProcessor/crocodile) to calculate the visibility data for a snapshot (i.e. single time sample) observation of a target field containing one point source. 

To run the script you will need:
 - ipython
 - numpy
 - pylab
 - VLA_A_hor_xyz.txt
 
The first three of these can be easily obtained using pip. 

To get pip: 

> apt-get install python-pip

To install each library:

>pip install numpy

etc.

The fourth is a text file supplied with this notebook. This script assumes it is in the working directory. 

In [ ]:
%matplotlib inline

In [ ]:
from __future__ import print_function
import numpy 
import matplotlib.pyplot as pl
import os
from arl.parameters import crocodile_path

The first thing we need to do is take the layout of our interferometer and convert it from Earth co-ordinates to UVW-coordinates, which are defined in the direction of our observation target. First, get the antenna positions:

In [ ]:
ants_xyz=numpy.genfromtxt(crocodile_path("data/vis/VLA_A_hor_xyz.txt"), delimiter=",")
print("Number of antennas in array:",ants_xyz.shape[0])

In [ ]:
pl.scatter(ants_xyz[:,0],ants_xyz[:,1])
pl.title("Antenna positions")
pl.xlabel(r"x [m]")
pl.ylabel(r"y [m]");

Then get the position of the target. This is given in Hour Angle and Declination:

In [ ]:
ha = 0.0; dec = numpy.pi/4.  # Units are Radians

With these inputs we can then calculate the UVW co-ordinates of the antennas:

In [ ]:
x,y,z=numpy.hsplit(ants_xyz,3)

t=x*numpy.cos(ha) - y*numpy.sin(ha)
u=x*numpy.sin(ha) + y*numpy.cos(ha)
v=-1.*t*numpy.sin(dec)+ z*numpy.cos(dec)
w=t*numpy.cos(dec)+ z*numpy.sin(dec)

ants_uvw = numpy.hstack([u,v,w])
print(ants_uvw)

(If we wanted a longer observation - more than a single snapshot - we would need to provide an array of Hour Angles and concatenate the baseline distributions for each HA increment. Declination remains fixed.)

From the positions of our antennas in UVW co-ordinates, we then can calculate the baseline distribution:

In [ ]:
res=[]
for i in range(ants_uvw.shape[0]):
    for j in range(i+1, ants_uvw.shape[0]):
        res.append(ants_uvw[j]-ants_uvw[i])

basel_uvw = numpy.array(res)
print("Number of baselines in array:", basel_uvw.shape[0])

We can plot this distribution:

In [ ]:
pl.subplot(111)
pl.scatter(basel_uvw[:,0],basel_uvw[:,1],c='b')
pl.title("Baseline distribution")
pl.xlabel(r"U [$\lambda$]")
pl.ylabel(r"V [$\lambda$]")
pl.show()

Note that we only have approximately half a plane of UVW points. Because the sky intensity will always be positive definite, we can infact mirror these samples using conjugate symmetry such that $$V(-{\mathbf u}) = V^{\ast}({\mathbf u}).$$ 

This means that our sampling actually looks like:

In [ ]:
pl.subplot(111)
pl.scatter(basel_uvw[:,0],basel_uvw[:,1],c='r')
pl.scatter(-1.*basel_uvw[:,0],-1.*basel_uvw[:,1],c='r')
pl.title("Mirrored Baseline distribution")
pl.xlabel(r"U [$\lambda$]")
pl.ylabel(r"V [$\lambda$]")
pl.show()

At the moment our target field is empty, i.e. we have zero signal. We can put a point source into our target field by defining it at a position relative to the target field centre $s_0 = (0,0,1)$. We define this position in terms of direction cosines $${\mathbf x_0} = (l, m, n),$$ and give our source an amplitude of $$S_0.$$

In [ ]:
l = 0.0005; m = 0.0000    # Units are Radians
src_pos=numpy.array([l, m , numpy.sqrt(1 - l**2 - m**2)-1])
src_amp = 100.0

The response of our interferometer to this source is given by 
$$V(\mathbf{u}) = \int{ S_0\, \delta({\mathbf x}-{\mathbf x_0})\,e^{-2i\pi\, {\mathbf u}\cdot ({\mathbf x} - s_0)}\, {\rm d}{\mathbf x}}$$

Note that we also rotate the visibility so it is relative to the phase centre $s_0$. `src_pos` is already corrected for this - note the "`-1`" above.

In [ ]:
vis = src_amp*numpy.exp(-2j*numpy.pi* numpy.dot(basel_uvw[:,0:3], src_pos[0:3]))

We can easily add additional sources just by repeating and summing up the visibilities:

In [ ]:
def add_source(l, m, src_amp):
    src_pos=numpy.array([l, m , numpy.sqrt(1 - l**2 - m**2)-1])
    return src_amp*numpy.exp(-2j*numpy.pi* numpy.dot(basel_uvw[:,0:3], src_pos[0:3])) 
vis += add_source(0.0000,0.0000,100)
vis += add_source(0.0015,0.0000,100)

These are continuous complex visibilities. They are currently noiseless.

We can write out these data:

In [ ]:
print(vis)
u = basel_uvw[:,0]
v = basel_uvw[:,1]
w = basel_uvw[:,2]
vis_re = vis.real
vis_im = vis.imag
numpy.savetxt('simulated_data.txt',numpy.column_stack((u,v,w,vis_re,vis_im)),newline='\n',fmt='%1.4e')

Done!